In [1]:
# Adversarial training using the ALUM algorithm overrides the usual training step method to generate the adversarial loss alongside the normal loss. Here, we demonstrate how to perform adversarial training and then evaluate using the primitive trainer class. Then, we show how this can be integrated into an automated pipeline using Flows.

In [2]:
# Load dummy train and test data from package
import pkg_resources
TRAIN_PATH = pkg_resources.resource_filename(
            'kitanaqa', 'support/unittest-squad.json')
EVAL_PATH = pkg_resources.resource_filename(
            'kitanaqa', 'support/unittest-squad.json')

In [3]:
from transformers import HfArgumentParser, TrainingArguments
from kitanaqa.trainer.arguments import ModelArguments
from kitanaqa.trainer.utils import load_and_cache_examples, build_flow
from kitanaqa.trainer.run_pipeline import _setup
hparams = {
            "model_type" : "distilbert",
            "model_name_or_path" : "distilbert-base-uncased",
            "output_dir" : "./example_outputs",
            "cache_dir" : "./example_outputs",
            "data_dir" : "./example_outputs",
            "train_file_path" : TRAIN_PATH,
            "predict_file_path" : {"dev-v1.1": EVAL_PATH},
            "aug_file_path" : None,
            "do_aug" : False,
            "do_alum" : True,
            "alpha" : 5,
            "eps" : 1e-4,
            "eta" : 1e-5,
            "sigma" : 1e-3,
            "do_train" : True,
            "do_eval" : True,
            "per_device_train_batch_size" : 2,
            "per_device_eval_batch_size" : 1,
            "gradient_accumulation_steps" : 2,
            "eval_all_checkpoints" : False,
            "num_train_epochs" : 1,
            "max_steps" : -1,
            "save_steps" : 1,
            "seed" : 512,
            "fp16" : False
}
# Initialize args
parser = HfArgumentParser(dataclass_types=[ModelArguments, TrainingArguments])
model_args, training_args = parser.parse_dict(hparams)
model, tokenizer, train_dataset = _setup(model_args, training_args)

11/12/2020 18:21:23 - kitanaqa - WARNING - Process rank: -1, device: cpu, n_gpu: 0, distributed training: False, 16-bits training: False
11/12/2020 18:21:23 - WARNING - kitanaqa -   Process rank: -1, device: cpu, n_gpu: 0, distributed training: False, 16-bits training: False
11/12/2020 18:21:23 - kitanaqa - INFO - Training/evaluation parameters TrainingArguments(output_dir='./example_outputs', overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=False, evaluate_during_training=False, prediction_loss_only=False, per_device_train_batch_size=2, per_device_eval_batch_size=1, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=2, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=1, max_steps=-1, warmup_steps=0, logging_dir='runs/Nov12_18-21-23_Aarons-MBP-7.attlocal.net', logging_first_step=False, logging_steps=500, save_steps=1, save_total_limit=None, no_cuda=Fa

11/12/2020 18:21:25 - INFO - filelock -   Lock 4832273912 released on ./example_outputs/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084.lock


11/12/2020 18:21:25 - INFO - filelock -   Lock 4832320704 acquired on ./example_outputs/a41e817d5c0743e29e86ff85edc8c257e61bc8d88e4271bb1b243b6e7614c633.8949e27aafafa845a18d98a0e3a88bc2d248bbc32a1b75947366664658f23b1c.lock


11/12/2020 18:21:25 - INFO - filelock -   Lock 4832320704 released on ./example_outputs/a41e817d5c0743e29e86ff85edc8c257e61bc8d88e4271bb1b243b6e7614c633.8949e27aafafa845a18d98a0e3a88bc2d248bbc32a1b75947366664658f23b1c.lock


11/12/2020 18:21:26 - INFO - filelock -   Lock 4832319920 acquired on ./example_outputs/ae9df7a8d658c4f3e1917a471a8a21cf678fa1d4cb91e7702dfe0598dbdcf354.c2015533705b9dff680ae707e205a35e2860e8d148b45d35085419d74fe57ac5.lock


11/12/2020 18:21:43 - INFO - filelock -   Lock 4832319920 released on ./example_outputs/ae9df7a8d658c4f3e1917a471a8a21cf678fa1d4cb91e7702dfe0598dbdcf354.c2015533705b9dff680ae707e205a35e2860e8d148b45d35085419d74fe57ac5.lock


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this mode

In [4]:
# Construct a Flow pipeline consisting of a parent step (alum training) conditionally followed by a child step (evaluation)
f = build_flow(
            (model_args, training_args),
            model=model,
            tokenizer=tokenizer,
            train_dataset=train_dataset)
if f:
    f.run()

[2020-11-13 02:21:51] INFO - prefect.FlowRunner | Beginning Flow run for 'default'


11/12/2020 18:21:51 - INFO - prefect.FlowRunner -   Beginning Flow run for 'default'


[2020-11-13 02:21:51] INFO - prefect.TaskRunner | Task 'train': Starting task run...


11/12/2020 18:21:51 - INFO - prefect.TaskRunner -   Task 'train': Starting task run...
/Users/asisto/dev/Searchable/Research/KitanaQA/env/lib/python3.7/site-packages/transformers-3.1.0-py3.7.egg/transformers/trainer.py:247: FutureWarning: Passing `prediction_loss_only` as a keyword argument is deprecated and won't be possible in a future version. Use `args.prediction_loss_only` instead.
  FutureWarning,
You are instantiating a Trainer but Tensorboard is not installed. You should consider installing it.


/Users/asisto/dev/Searchable/Research/KitanaQA/src/kitanaqa/trainer/train.py:231: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self._delta = torch.tensor(sample, requires_grad = True, device = self.args.device)
/Users/asisto/dev/Searchable/Research/KitanaQA/env/lib/python3.7/site-packages/torch-1.5.1-py3.7-macosx-10.9-x86_64.egg/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


...

11/12/2020 18:22:36 - INFO - prefect.TaskRunner -   Task 'eval': Starting task run...
11/12/2020 18:22:36 - kitanaqa.trainer.utils - WARNING - You are running a non-local model checkpoint. This may or may not be what you intended.
11/12/2020 18:22:36 - WARNING - kitanaqa.trainer.utils -   You are running a non-local model checkpoint. This may or may not be what you intended.
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the check

[2020-11-13 02:22:44] INFO - prefect.TaskRunner | Task 'eval': finished task run for task with final state: 'Success'


11/12/2020 18:22:44 - INFO - prefect.TaskRunner -   Task 'eval': finished task run for task with final state: 'Success'


[2020-11-13 02:22:44] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded


11/12/2020 18:22:44 - INFO - prefect.FlowRunner -   Flow run SUCCESS: all reference tasks succeeded


In [ ]:
f.result